In [1]:
import sys
import numpy as np
from matplotlib import pyplot as plt
import h5py

%matplotlib notebook

In [2]:
import qcodes as qc

# Setup notes

For this example to run, you need to have set up a labpython config file. There is a template in labpythonQDelft/templates.

In [14]:
from importlib import reload
from qctools import instruments as instools
reload(instools)

import labpythonconfig as cfg
reload(cfg)

from experiment import measurement
reload(measurement)
from experiment.measurement import Parameter, BaseMeasurement

In [15]:
from pysweep.data_storage import np_storage
reload(np_storage)
from pysweep.data_storage.np_storage import NpStorage, DataSet

In [16]:
station = qc.Station()

In [20]:
class DummyMeasurement(BaseMeasurement):
    def __init__(self, station):
        super().__init__(station)
        
        self.add_parameter('start', Parameter)
        self.add_parameter('stop', Parameter)
        self.add_parameter('npts', Parameter)
    
    def measure(self):        
        # set up loop
        xpoints = np.linspace(self.start(), self.stop(), self.npts())
        ypoints = np.linspace(self.start(), self.stop(), self.npts())
                
        # now populate in a primitive loop
        for x in xpoints:
            for y in ypoints:
                
                # acquire some data
                z = np.cos(x) * np.cos(y) * np.linspace(0,1,3)

                # add to dataset
                self.data.add(
                    {'x' : {'value' : x, 'independent_parameter' : True, 'unit' : 'V'},
                     'y' : {'value' : y, 'independent_parameter' : True, 'unit' : 'V'},
                     'z' : {'value' : z },
                    }
                )
                    

m = DummyMeasurement(station)
m.start(0)
m.stop(10)
m.npts(3)
m.run()

In [ ]:
voltage1 = 0
voltage2 = 0

def getv1():
    global voltage1
    return voltage1

def setv1(v):
    global voltage1
    voltage1 = v

def getv2():
    global voltage2
    return voltage2

def setv2(v):
    global voltage2
    voltage2 = v

v1 = qc.Parameter('voltage1', get_cmd=getv1, set_cmd=setv1)
v2 = qc.Parameter('voltage2', get_cmd=getv2, set_cmd=setv2)

station = qc.Station(v1, v2)

In [ ]:
from pysweep import sweep_object
reload(sweep_object)
from pysweep.sweep_object import sweep, ChainSweep

class PysweepMeasurement(BaseMeasurement):
    """
    A simple prototype for how we could write a measurement around pysweep loops.
    
    The idea here is that when inheriting from this, the user only needs to specify the
    sweep and acquire methods. The measure methods should just work for all sweeps.
    Importantly, the data allocation is taken out of the user's hands here.
    """
    
    def measure(self):
       
        # knowing the dimension allows us to allocate a dataset
        # this only works for scalar data.
        # if the acquire methods returns an array, then we would have to wait
        # with allocation until we know the returned shape (depends on class implementation)
        data = xr.DataArray(np.zeros(shp), coords=coords)
        self.dataset['data'] = data
        self.save_data()
        
        # this is not very elegant, but since i don't really know 'where' in the
        # sweep pysweep currently is, this is the easiest i could come up with.
        # this line generates the n-d array indices in the order that the sweep loops through the coordinates.
        idxs = zip(*[ idx.reshape(-1) for idx in np.meshgrid(*[np.arange(c[1].size) for c in coords], indexing='ij') ])
        
        for idx, data in zip(idxs, sweep):
            data_val = data['data']['value']
            print(idx, self.station.voltage1(), self.station.voltage2(), data_val)
            
            self.dataset['data'][idx] = data_val
            self.save_data()
        
    
    # These are examples -- will be implemented by the user.
    def acquire(self, station, namespace):
        return {'data' : {'value' : self.station.voltage1() * self.station.voltage2()**2 }}
        
    def sweep(self):
        return ChainSweep([
            sweep(self.station.voltage1, np.linspace(0,1,3)) (
                sweep(self.station.voltage2, np.linspace(0,2,5)) (
                    self.acquire,
                )
            )
        ])
    

m = SimplePysweepMeasurement(station)
m.run()

In [ ]:
fn = m.datafilepath
print(fn)

In [ ]:
with xr.open_dataset(fn) as ds:
    z = ds['data'].values
    v1 = ds['data'].coords['voltage1'].values
    v2 = ds['data'].coords['voltage2'].values

In [ ]:
fig, ax = plt.subplots(1,1)
for i, v1val in enumerate(v1):
    ax.plot(v2, z[i, :], '-o', label='v1 = {}'.format(v1val))

ax.legend()
ax.set_xlabel("v2")

In [20]:
[i if i<3 else i*2 for i in range(5)]

[0, 1, 2, 6, 8]